Test loader examples
Ref https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/


In [1]:
import os
from qdrant_client import QdrantClient
from openai import OpenAI


In [2]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient

In [3]:
api_key=os.getenv("OPENAI_API_KEY")
docs_dir="data/"
file_name="Databricks-Big-Book-Of-GenAI-FINAL.pdf"
qdrant_url="http://localhost:6333/"
qdrant_collection="databricks-big-book"
strategy="fast"

Define embedding function (openai) and qdrant client

In [4]:
embeddings = OpenAIEmbeddings(
            openai_api_key=api_key
        )

qdrant_client = QdrantClient(url=qdrant_url,
                             port=None,
                             api_key=""
                             )

/home/mcastrol/github/openllmmodels/.venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


1) Using Unstructured File loader 

In [5]:
loader = UnstructuredFileLoader(f"{docs_dir}{file_name}",  mode="elements")
docs = loader.load()
records=len(docs)


/home/mcastrol/github/openllmmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
qdrant_collection=f"{qdrant_collection}_unstructured"
print(f"Hay {records} documentos a cargar en la colleccion {qdrant_collection}")
qdrant_client.delete_collection(qdrant_collection)

Hay 2741 documentos a cargar en la colleccion databricks-big-book_unstructured


True

Saving docs in vector database

In [7]:
docs[0]

Document(page_content='CONTENTS', metadata={'source': 'data/Databricks-Big-Book-Of-GenAI-FINAL.pdf', 'coordinates': {'points': ((36.0, 120.55960000000005), (36.0, 160.55960000000005), (246.368, 160.55960000000005), (246.368, 120.55960000000005)), 'system': 'PixelSpace', 'layout_width': 1024.0, 'layout_height': 768.0}, 'file_directory': 'data', 'filename': 'Databricks-Big-Book-Of-GenAI-FINAL.pdf', 'languages': ['eng'], 'last_modified': '2024-05-13T14:35:55', 'page_number': 2, 'filetype': 'application/pdf', 'category': 'Title'})

In [8]:
Qdrant.from_documents(
    docs,
    embeddings,
    url="http://localhost:6333/",
    api_key="",
    port=None,
    collection_name=qdrant_collection
)

/home/mcastrol/github/openllmmodels/.venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")
/home/mcastrol/github/openllmmodels/.venv/lib/python3.11/site-packages/qdrant_client/async_qdrant_remote.py:117: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


2.- Using PyPDFLoader

In [9]:
from langchain_community.document_loaders import PyPDFLoader

file_path = f"{docs_dir}{file_name}"
print(file_path)
loader = PyPDFLoader(file_path)
docs = loader.load_and_split()
records=len(docs)

data/Databricks-Big-Book-Of-GenAI-FINAL.pdf


In [10]:
qdrant_collection=qdrant_collection+"_pdfloader"
print(f"Hay {records} documentos a cargar en la colleccion {qdrant_collection}")
qdrant_client.delete_collection(qdrant_collection)

Hay 118 documentos a cargar en la colleccion databricks-big-book_unstructured_pdfloader


False

In [ ]:
Qdrant.from_documents(
    docs,
    embeddings,
    url="http://localhost:6333/",
    api_key="",
    port=None,
    collection_name=qdrant_collection
)